# Tutorial 3: Creating a power production forecast for a wind turbine

**Content creators**: Fabian Emmerich

**Content reviewers / testers**: Jannik Jauch

In this tutorial, you will be creating a power production forecast for a wind turbine from realistic production data. In Tutorial 2, you have worked yourself through the data produced by a numerical weather model and a wind turbine. Now, you will learn how to extract features from these data that are required to produce the production forecast, and you will actually train and apply a model that has the ability to produce the forecast.

## Exercise 1: Preparing the target variable

In our concrete case, the so-called `target variable` of our problem is the power production of the wind turbine. It is that quantity we want to predict with the model we will train.

However, these data are not yet in the correct shape:

1. As you have seen in Tutorial 2, the data contain outliers. These are extreme cases where the production is either irregularly high or low, or where it is 0 although there is a non-negligible wind speed measured.
1. The data is sampled in a lower resolution than the weather data. The weather data have an hourly resolution, whereas the power production is measured in 10-minute intervals.

Hence, we first have to adapt the production data to match the other input data for the model.

### Tasks

1. Remove all data samples that can be considered as outliers (or noise), e.g. such that
   1. exhibit extreme production values (e.g. such where $P \gtrsim 1000\,\mathrm{kW}$ or $P \lesssim 0\,\mathrm{kW}$) and
   1. have a production $P = 0\,\mathrm{kW}$.
   1. where the production is `NaN`.
1. Get an impression of how the cleaned dataset looks.
   1. Plot the wind speed vs. power production scatter plot before and after to see if you have removed the desired samples. You may also confirm your procedure by looking at histogram plots before and after.
   1. How much of the data got "lost" by the previous step?
1. Resample the data to an hourly resolution. 

   *Hints:* 
     - E.g. calculate the mean for each resampled time step.
     - Remember that you have removed `NaNs` in Task 1.
       Thus, resampling time invervals that are not missing will
       create `NaNs` again that have to be removed as well.

In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-3/task-1-1.py

In [ ]:
# Solution to Task 2
%load ./solutions/tutorial-3/task-1-2.py

In [ ]:
# Solution to Task 3
%load ./solutions/tutorial-3/task-1-3.py

## Exercise 2: Selecting suitable features

The most important aspect in ML is data. Especially the size of a dataset as well as the data quality is most important. However, this doesn't necessarily mean that utilizing more features to train a model will increase the model quality (forecast quality). On the contrary, this may actually lead to overfitting and the model quality may get worse.

A good example to illustrate this is extreme cases - or outliers. Typically, you want to avoid having them in your dataset, because they reduce the quality of the dataset, i.e. they increase the noise. 

Imagine you have a dataset about persons with the features age, size, and weight. This already gives us a 3D feature space. You aim to solve some classification problem with these data, i.e. you now want to find groups of people that share similar properties. Looking at each feature individually, you will have very natural Gaussian distributions given a large enough dataset size. In the full 3D feature space, these will still show a dense distribution, where e.g. size and weight will likely show a strong positive correlation. There will be a couple of outliers, although the vast majority of the samples are within the expected range. 

Now imagine adding more features, maybe even very bizarre features that you think might help finding groups of connected people. Maybe hair and eye color might help with your classification problem, or shoe size, or how much money each individual spend on grocieries or hygiene products, and so forth. Adding a new feature, though, also means increasing the dimension of the feature space. With each feature you increase the likelihood that any given data sample might become an outlier in any of these dimensions. As a consequence, this increases the amount of noise and makes the dataset more sparse. When training a model, this might not even become well visibile. This effect, however, is most crucial when it comes to inference with unknown data samples, because it is even harder to cope with unknown samples that are potentially far away from known training instances.

A less abstract and more mathemtical explanation of this effect is the distance of two random points in an $N$-dimensional unit square. For $N=2$, the average distance is $\sim 0.52$. For $N=3$ (a unit cube), it becomes $0.66$. For $N \rightarrow \infty$ (an $N$-dimensional hypercube), this distance increases further. For $N=10^6$, for instance, this average distance becomes $\sim 408.25$. Hence, high-dimensional datasets are at risk of being very sparse.

This phenomenon is known as the *Curse of Dimensionality*.

**TL;DR:** Be careful when selecting the features for training your model. Including too many features will make the data sparse and, counterintuitively, maybe even reduce the model performance.

Hence, we will now take a look at the features of the weather and select only those which we might think are the best fit for our problem.

### Choosing a subsample from the data

However, we still have a problem: our weather model covers a very large grid on multiple altitudes (levels). Adding the entire grid and all altitudes would make the number of dimensions explode. Hence, we need to reduce the amount of input data in advance.

For reasons of simplicity, we will only pick the data from a single grid cell which is closest to the geological position of the wind turbine.

Regarding the altitudes, we also need to make a choice which one to pick. Since the phyiscal quantities can vary on different levels due to distinct air layers in the atmosphere, it makes sense to pick the model level which is closest to the hub height of the wind turbine. However, these are not fixed but depend on model level temperature and absolute humidity, and surface pressure (see [here](https://confluence.ecmwf.int/display/ECC/Compute+geopotential+on+model+levels)), and, thus, actually would need to be computed for every time step and grid point.

For reasons of simplicity, though, we will here use data only from the model level $L = 133$ that usually corresponds to $\sim 100\,\mathrm{m}$, which is a typical hub height of a turbine.

The dataset is available at `/p/project/training2223/a6/data/ml_level_133_2017_2020.nc`.

### Tasks
1. Select the data only from the grid point that is closest to the wind turbine's geological location.
1. Taking a look at the time ranges of the data you might notice that they don't cover identical time ranges. And, as you remember correctly, we have removed a lot of outliers from the wind turbine data. Thus, we have to take all data points in each dataset that have identical time steps. Once done, you can merge them into a single dataset.

   *Hints:* 
     - Python's [`set`](https://realpython.com/python-sets/) is a perfect data structure for set theory operations.
     - For merging, you should select the level present in each dataset and override the `level` value when merging, since both dataset have different levels.
1. Investigate the features of the weather data at that grid point. Look how well every feature correlates with one another.
   1. Create correlation plots for each quantity. Besides the correlation to the target variable, also check for correlation between each quantity.
   1. Calculate the Pearson correlation coefficient for each quantity.
   
      *Hint:* `pandas.DataFrame` allows a direct calculation of the correlation matrix (`pandas.DataFrame.corr()`). You can then also use the `utils.create_heatmap_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook to visualize the resulting correlation matrix.
   1. Based on the two above criteria, select only those features that you would consider valuable as input for model training.

In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-3/task-2-1.py

In [ ]:
# Solution to Task 2
%load ./solutions/tutorial-3/task-2-2.py

In [ ]:
# Solution to Task 3
%load ./solutions/tutorial-3/task-2-3.py

## Exercise 3: Feature engineering

In the above tasks, you have concluded that there is a correlation between some of the given output quantities of the numerical weather model and the production of the wind turbine. 

However, it may also be the case that there are new features extractable from the given quantities that also show a similar or even better correlation to the target variable. By doing so, you may also reduce the dimension of the feature space through melting related features into new input features. For example, if you use 3 well-correlated features to calculate a new input feature that shows a similar correlation, you have reduced the feature space by 2 dimensions. As a side effect, this also reduces the amount of data you need for training.

In our concrete case, you might find such features also based on phyiscal reasoning. Which effects might affect the power production of a wind turbine?

However, we found that it might be sufficient to only use the absolute wind speed to train a simple model that will likely peform well enough.

### Tasks

1. Calculate the total wind speed.

   *Hint:* You can cast `numpy` methods directly onto `xr.DataArray`.

In [ ]:
# Solution to Task 1
%load ./solutions/tutorial-3/task-3-1.py

## Exercise 4: Training a model and creating a production forecast

Now, we want to train a model from our feature(s). We've seen, that there is a strong linear correlation to wind speed, but in fact it is not entirely linear. Hence, we might not just want to use a linear regression model, but something that is also able to handle non-linear correlations.

Typically, production forecasts are provided for up to a several days in e.g. an hourly temporal resolution. A good measure to evaluate the preciseness of a forecast is the [mean absolute error (MAE)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html).

### Task

1. Check the `sklearn` library for a model that is able to handle data with non-linear relationships.
1. Now, create a model, make a forecast, and evaluate the result:
   1. Split the dataset into two sets of train and test samples. Choose a suitable size for your test size.
   1. Train the model with the train data.
   1. Create a forecast with the test data.
   1. Calculate the MAE. What is the unit of the MAE and how can it be interpreted?
   1. Now calculate the normalized MAE ($\mathrm{NMAE} = \mathrm{MAE}/P_\mathrm{turbine}~~~$, where $P_\mathrm{turbine}~~$ is the power rating (i.e. maximum power output) of the wind turbine). How can this be interpreted? Assume a power rating of 850 kW.
   1. Plot the forecast and the real production data.
   
      *Hint:* You can use the `utils.create_production_and_forecast_comparison_plot()` method from the [`utils`](./utils.py) package located in the folder of this notebook.
1. _(Optional)_ Tune your model or try out some other models and evaluate their performance on the given problem.

In [ ]:
# Solution to Task 2
%load ./solutions/tutorial-3/task-4-2.py